In [27]:
import geopandas as gpd
import pandas as pd
import numpy as np

from skimage.io import imread
from skimage.morphology import square
from skimage.filters.rank import majority
from rasterio.mask import mask
import rasterio as rio

from osgeo import gdal, ogr, osr
gdal.PushErrorHandler('CPLQuietErrorHandler')
gdal.UseExceptions()

from tqdm.notebook import tqdm

from concurrent.futures import ThreadPoolExecutor, as_completed
import glob
import os

In [28]:
ls_folder_result=['E_Wida','G_Muhlis','H_Andi','I_Stefanus','J_Fawcet','L_Linda']

In [29]:
daratan_ref=gpd.read_file('../shared_pemetaan/google-cloud/vector_references/daratan_bts.gpkg').to_crs('EPSG:3857')
sawit_ref=gpd.read_file('../shared_pemetaan/google-cloud/vector_references/sawit.gpkg').to_crs('EPSG:3857')
hutan_ref=gpd.read_file('../shared_pemetaan/google-cloud/vector_references/hutan.gpkg').to_crs('EPSG:3857')
tambak_ref=gpd.read_file('../shared_pemetaan/google-cloud/vector_references/tambak.gpkg').to_crs('EPSG:3857')

In [30]:
def post_class(i,folder_loc):
    mj_fil1=square(5)
    mj_fil2=square(3)
    data=imread(i, plugin='tifffile')
    t_0=pd.DataFrame(np.asarray(np.unique(data,return_counts=True)).T,columns=['class','init'])
    major_data=majority(data.astype(np.uint8),mj_fil1)
    major_data=majority(major_data,mj_fil2)
    t_1=pd.DataFrame(np.asarray(np.unique(major_data,return_counts=True)).T,columns=['class','after_MFilter'])
    with rio.open(i) as src:
        out_image, out_transform = mask(src, daratan_ref.geometry,invert=True)
        water_masked=out_image[0]
        water_masked=np.ma.masked_where(water_masked>0,water_masked).mask
        out_image, out_transform = mask(src, tambak_ref.geometry)
        tambak=out_image[0]
        tambak=np.ma.masked_where(tambak>0,tambak).mask
        out_image, out_transform = mask(src, hutan_ref.geometry)
        hutan_masked=out_image[0]
        hutan_masked=np.ma.masked_where(hutan_masked>0,hutan_masked).mask
        out_image, out_transform = mask(src, sawit_ref.geometry)
        sawit=out_image[0]
        sawit=np.ma.masked_where(sawit>0,sawit).mask
    major_data[water_masked]=11
    t_2=pd.DataFrame(np.asarray(np.unique(major_data,return_counts=True)).T,columns=['class','after_RB_water'])
    major_data[tambak]=10
    t_3=pd.DataFrame(np.asarray(np.unique(major_data,return_counts=True)).T,columns=['class','after_RB_tambak'])
    major_data[hutan_masked]=5
    t_4=pd.DataFrame(np.asarray(np.unique(major_data,return_counts=True)).T,columns=['class','after_RB_hutan'])
    major_data[sawit]=31
    t_5=pd.DataFrame(np.asarray(np.unique(major_data,return_counts=True)).T,columns=['class','after_RB_sawit'])
    with rio.open(i) as src:
        out_meta = src.meta
        out_meta.update({'driver':'GTiff',
                    'height':src.shape[0],
                    'width':src.shape[1],
                    'transform':src.transform})
    with rio.open(folder_loc+'/post_t_'+i[-22:],'w',**out_meta) as dest:
        dest.write(major_data,1)
    Image = gdal.Open(folder_loc+'/post_t_'+i[-22:], 1)  # open image in read-write mode
    gt=Image.GetGeoTransform()
    rows = Image.RasterYSize
    cols = Image.RasterXSize
    Band = Image.GetRasterBand(1)
    proj_ref = Image.GetProjectionRef()
    driver1 = gdal.GetDriverByName('GTiff')
    outRaster = driver1.Create(folder_loc+'/post_'+i[-22:], cols, rows, 1, gdal.GDT_UInt16)
    outRaster.SetProjection(proj_ref)
    outRaster.SetGeoTransform(gt)
    outband = outRaster.GetRasterBand(1);
    gdal.SieveFilter(srcBand=Band, maskBand=None, dstBand=outband, 
                 threshold=5, connectedness=4, callback=None)
    del Image, Band, outRaster, outband
    data_out=imread(folder_loc+'/post_'+i[-22:], plugin='tifffile')
    t_6=pd.DataFrame(np.asarray(np.unique(data_out,return_counts=True)).T,columns=['class','fin'])
    t_=t_0.merge(t_1,how='left').merge(t_2,how='left').merge(t_3,how='left').merge(t_4,how='left').merge(t_5,how='left').merge(t_6,how='left')
    t_.to_csv(folder_loc+'/post_'+i[-22:-3]+'csv',index=False)
    os.remove(folder_loc+'/post_t_'+i[-22:])
    os.remove(folder_loc+'/post_'+i[-22:])
    data_out=data_out.astype(np.uint8)
    with rio.open(i) as src:
        out_meta = src.meta
        out_meta.update({'driver':'GTiff',
                    'height':src.shape[0],
                    'width':src.shape[1],
                    'transform':src.transform})
    with rio.open(folder_loc+'/post_'+i[-22:],'w',**out_meta) as dest:
        dest.write(data_out,1)

In [ ]:
for folder_result in ls_folder_result:
    glob_result=glob.glob('../shared_pemetaan/google-cloud/classification_result/'+folder_result+'/*.tif')
    glob_post=glob.glob('../shared_pemetaan/google-cloud/post_classification_result/'+folder_result+'/*.tif')
    check=[i for i in glob_result if '../shared_pemetaan/google-cloud/post_classification_result/'+folder_result+'/post_'+i[-22:] not in glob_post]
    for i in tqdm(check):
        post_class(i,'../shared_pemetaan/google-cloud/post_classification_result/'+folder_result+'/')

  0%|          | 0/427 [00:00<?, ?it/s]